# Токенизация

Как разделить текст на предложения?

In [ ]:
text = "Mr. Smith bought ticket to San Francisco. He was very happy."
text.split('.')

In [ ]:
#!pip install nltk

In [ ]:
#nltk.download('punkt')

In [ ]:
import nltk

In [ ]:
nltk.sent_tokenize(text)

## По пробелам

In [ ]:
en_sentence = nltk.sent_tokenize(text)[0]
en_sentence.split()

In [ ]:
ru_sentence = "Мистер Смит купил билет до Сан-Франциско."
ru_sentence.split()

## NLTK

In [ ]:
nltk.word_tokenize(en_sentence)

In [ ]:
nltk.word_tokenize(ru_sentence)

## pymorphy2

In [ ]:
import pymorphy2
m = pymorphy2.MorphAnalyzer()
for t in nltk.word_tokenize(ru_sentence):
    print(m.parse(t)[0])

In [ ]:
for t, p in nltk.pos_tag(nltk.word_tokenize(en_sentence)):
    print(t, p)

## Вопрос на подумать
А что делать с языками без пробелов?

...

# BPE / Subword segmentation

In [ ]:
!pip3 install subword-nmt

Посмотрим по шагам на обучение BPE-токенизации:

увеличивая количество операций (`-s`), получаем новые токены в _словаре_.

In [ ]:
!subword-nmt learn-bpe -s 1 < train.txt > bpe_result.txt

In [ ]:
!echo "newer lower" | subword-nmt apply-bpe -c bpe_result.txt

# Toy language model
Научимся быстро считать частоты токенов/n-грамм в корпусе:

In [ ]:
from collections import Counter

In [ ]:
corpus = """
Лингви́стика (от лат. lingua «язык»), языкозна́ние, языкове́дение — наука, изучающая языки.
Это наука о естественном человеческом языке вообще и обо всех языках мира как его индивидуализированных представителях.
В широком смысле слова лингвистика подразделяется на научную и практическую. Чаще всего под лингвистикой подразумевается именно научная лингвистика. Является частью семиотики как науки о знаках.
"""

In [ ]:
tokenized_corpus = nltk.word_tokenize(corpus)

Как получить биграммы и триграммы

In [ ]:
# Приводим к списку, потому что nltk возвращает генератор
list(nltk.bigrams(tokenized_corpus))

In [ ]:
list(nltk.trigrams(tokenized_corpus))

Посчитать количество каждого элемента в списке - Counter

In [ ]:
unigrams = Counter(tokenized_corpus)

In [ ]:
unigrams

Выведем топ по частоте

In [ ]:
unigrams.most_common(10)

## Нормализуем частоты
Чтобы получить вероятности униграмм

In [ ]:
n = len(tokenized_corpus)
for k in unigrams:
    unigrams[k] /= n

In [ ]:
unigrams.most_common(10)

Оценим вероятность тестового предложения:

In [ ]:
test_sentence = nltk.word_tokenize("Лингвистика - это наука о языке.")

In [ ]:
p = 1.
for token in test_sentence:
    p *= unigrams[token]
print(p)

!! Почему тут 0?

Применим простейшее сглаживание:

In [ ]:
p = 1.
for token in test_sentence:
    if unigrams[token]:
        p *= unigrams[token]
    else:
        p *= 1 / n
print(p)

Приведем вероятность к более удобочитаемому виду:

In [ ]:
from math import log
log(p)